In [2]:
// Helper functions
const log = <T extends unknown[]>(...data: T) => (console.log(...data), data);
const mod = (a: number, b: number) => ((a % b) + b) % b;

In [3]:
// Preprocess data
const exampleRawData = `
7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3
`.slice(1, -1);

const rawData = await Deno.readTextFile("day_9.txt");

function preprocess(rawData: string) {
  return rawData
    .split("\n")
    .map((e) => e.split(",").map(Number) as [number, number]);
}

In [4]:
function part1(data: ReturnType<typeof preprocess>) {
  let max = 0;
  for (let i = 0; i < data.length - 1; i++) {
    const a = data[i];
    for (let j = i + 1; j < data.length; j++) {
      const b = data[j];
      const area = Math.abs(a[0] - b[0] + 1) * Math.abs(a[1] - b[1] + 1);
      max = area > max ? area : max;
    }
  }
  return max;
}

part1(
  preprocess(
    // exampleRawData,
    rawData,
  )
)

4741848414

In [15]:
// function part2(data: ReturnType<typeof preprocess>) {
//   let max = 0;
//   for (let i = 0; i < data.length - 1; i++) {
//     const a = data[i];
//     x: for (let j = i + 1; j < data.length; j++) {
//       const b = data[j];
//       const area = (Math.abs(a[0] - b[0]) + 1) * (Math.abs(a[1] - b[1]) + 1);
//       if (area > max) {
//         const minX = Math.min(a[0], b[0]);
//         const maxX = Math.max(a[0], b[0]);
//         const minY = Math.min(a[1], b[1]);
//         const maxY = Math.max(a[1], b[1]);
//         // Check no points in this rectangle
//         for (let k = 0; k < data.length; k++) {
//           const [x, y] = data[k];
//           if (x > minX && x < maxX && y > minY && y < maxY) continue x;
//         }
//         max = area;
//         console.log(a, b, area);
//       }
//     }
//   }
//   return max;
// }
function part2(data: ReturnType<typeof preprocess>) {
  // Minimize x and y's, tightening bounds
  const xMap = [-1, ...[...new Set(data.map(e => e[0]))].sort((a,b) => a-b),100_000];
  const yMap = [-1, ...[...new Set(data.map(e => e[1]))].sort((a,b) => a-b),100_000];
  const xMapRev = Object.fromEntries(xMap.map((e,i) => [e,i]));
  const yMapRev = Object.fromEntries(yMap.map((e,i) => [e,i]));

  const w = xMap.length;
  const h = yMap.length;
  const grid = new Array(w*h).fill(0);

  data = data.map(([x,y]) => [xMapRev[x], yMapRev[y]]);
  
  for (let i = 0; i < data.length; i++) {
    const a = data[i];
    const b = data[(i + 1) % data.length];
    if (a[0] === b[0]) {
      const minY = Math.min(a[1], b[1]);
      const maxY = Math.max(a[1], b[1]);
      for (let y = minY; y <= maxY; y++) {
        const i = y * w + a[0];
        grid[i] = 1;
      }
    } else {
      const minX = Math.min(a[0], b[0]);
      const maxX = Math.max(a[0], b[0]);
      for (let x = minX; x <= maxX; x++) {
        const i = a[1] * w + x;
        grid[i] = 1;
      }
    }
  }
  function fill(i = 0) {
    const stack = [i];
    while (stack.length) {
      const i = stack.pop()!;
      grid[i] = 2;
      if (grid[i + 1] === 0) stack.push(i + 1);
      if (grid[i - 1] === 0) stack.push(i - 1);
      if (grid[i + w] === 0) stack.push(i + w);
      if (grid[i - w] === 0) stack.push(i - w);
    }
  }
  fill()
  console.log(Array(h).fill(0).map((_,i) => grid.slice(i * w, i * w + w).join("")).join("\n"))

  
  let max = 0;
  for (let i = 0; i < data.length - 1; i++) {
    const a = data[i];
    x: for (let j = i + 1; j < data.length; j++) {
      const b = data[j];
      const area = (Math.abs(xMap[a[0]] - xMap[b[0]]) + 1) * (Math.abs(yMap[a[1]] - yMap[b[1]]) + 1);
      if (area > max) {
        const minX = Math.min(a[0], b[0]);
        const maxX = Math.max(a[0], b[0]);
        const minY = Math.min(a[1], b[1]);
        const maxY = Math.max(a[1], b[1]);
        // Check no points in this rectangle
        for (let y = minY; y <= maxY; y++) {
          for (let x = minX; x <= maxX; x++) {
            const i = y * w + x;
            if (grid[i] === 2) continue x;
          }
        }
        max = area;
        // console.log(a, b, area);
        console.log([xMap[a[0]], yMap[a[1]]], [xMap[b[0]], yMap[b[1]]], area);
      }
    }
  }
  return max;
}

part2(
  // preprocess(exampleRawData)
  preprocess(rawData)
)


...
[ 97777, 50438 ] [ 97777, 51654 ] 1217
[ 97777, 50438 ] [ 97408, 54048 ] 1336070
[ 97777, 50438 ] [ 2347, 51978 ] 147059171
[ 97408, 54048 ] [ 3007, 58105 ] 383083316
[ 6073, 67455 ] [ 94582, 50408 ] 1508918480


1508918480